# FONTE DOS DADOS DO SATÉLITE GOES-16:

----------------------------------------------------

* **`[canal do visível (TVIS, 0,64 μm, 500 m)]`** - canal 2: http://ftp.cptec.inpe.br/goes/goes16/retangular/ch02/
* Exemplo: S10635334_202108010810.nc

----------------------------------------------------

* **`[canal do vapor d’água (TW, 6,15 μm, 2 km)]`** - canal 8: http://ftp.cptec.inpe.br/goes/goes16/retangular/ch08/
*    Exemplo: S10635340_202006291440.nc

----------------------------------------------------

* **`[canal do infravermelho (TIR, 10,3 μm, 2 km)]`** - canal 13: http://ftp.cptec.inpe.br/goes/goes16/retangular/ch13/  
* Exemplo: S10635346_202007010000.nc

----------------------------------------------------

# **1° Passo:** Importando as bibliotecas

In [ ]:
import proplot as pplt
import xarray as xr
import numpy as np
from matplotlib.colors import LinearSegmentedColormap 
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import os
import glob
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import time
from matplotlib import cm                          
import salem
from cpt_convert import loadCPT

import warnings
warnings.filterwarnings("ignore")

# **2° Passo:** Declarando funções

In [ ]:
#-----------------------------------------------------------------------------------
# Função que plota os Estados
#-----------------------------------------------------------------------------------
def evm_plot_states(shapefile, cor, espessura_linha):

    """
    Retorna a plotagem dos contornos de um shapefile na figura

    Parâmetros de entrada:
                shapefile (shp): shapefile da regiao

    Parâmetros de saída:
                figura com o contorno da região baseado no shapefile fornecido
    """
    import cartopy.crs as ccrs
    import cartopy.io.shapereader as shpreader
    import matplotlib.pyplot as plt

    shapefile = list(shpreader.Reader(shapefile).geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor=cor,facecolor='none', linewidth=espessura_linha)

#-----------------------------------------------------------------------------------
#  Função que plota circulos de distância em geral
#-----------------------------------------------------------------------------------
def evm_plota_aneis_em_geral(aneis, lon_r, lat_r, color, label, linestyle):

    """
    Retorna círculos de distância centrado no radar

    Parâmetros de entrada:
                aneis (lista): tamanho do raio do círculo em km
                lon_r (float): valor da longitude do centro do círculo em graus
                lat_r (float): valor da latitude do centro do círculo em graus
                color (str): cor do raio do círculo
                label (str): legenda

    Parâmetros de saída:
                latitude e longitude que delimita círculos de distância centrado no radar e plota os círculos
    """

    import geopy
    from geopy import distance

    origin = geopy.Point(lat_r, lon_r)

    lons = np.zeros((len(aneis), 361))
    lats = np.zeros((len(aneis), 361))
    for i, dis in enumerate(aneis):
        xpts = []
        ypts = []
        for az in range(361):
            destination = distance.distance(kilometers=dis).destination(origin, az)
            lat2, lon2 = destination.latitude, destination.longitude
            xpts.append(lon2)
            ypts.append(lat2)
        lons[i,:] = xpts[:]
        lats[i,:] = ypts[:]

    for i, anel in enumerate(aneis):
        ax.plot(lons[i,:], lats[i,:], color=color, label= label, linestyle=linestyle)

#-----------------------------------------------------------------------------------
#  Função que plota circulos de distância em geral
#-----------------------------------------------------------------------------------
def evm_plota_aneis_em_geral_painel(aneis, lon_r, lat_r, color, label, linestyle, axx, lww):

    """
    Retorna círculos de distância centrado no radar

    Parâmetros de entrada:
                aneis (lista): tamanho do raio do círculo em km
                lon_r (float): valor da longitude do centro do círculo em graus
                lat_r (float): valor da latitude do centro do círculo em graus
                color (str): cor do raio do círculo
                label (str): legenda

    Parâmetros de saída:
                latitude e longitude que delimita círculos de distância centrado no radar e plota os círculos
    """

    import geopy
    from geopy import distance

    origin = geopy.Point(lat_r, lon_r)

    lons = np.zeros((len(aneis), 361))
    lats = np.zeros((len(aneis), 361))
    for i, dis in enumerate(aneis):
        xpts = []
        ypts = []
        for az in range(361):
            destination = distance.distance(kilometers=dis).destination(origin, az)
            lat2, lon2 = destination.latitude, destination.longitude
            xpts.append(lon2)
            ypts.append(lat2)
        lons[i,:] = xpts[:]
        lats[i,:] = ypts[:]

    for i, anel in enumerate(aneis):
        ax[axx].plot(lons[i,:], lats[i,:], color=color, label= label, linestyle=linestyle, lw=lww)


# **Plota figura** - `SEM CÍRCULOS DOS RADARES`

In [ ]:
%%time
#==================================================================================================#
#                              DEFINIÇÃO DE PARÂMETROS
#==================================================================================================#
# corees do IR
cpt_ir = loadCPT('ir.cpt')
cmap_ir = cm.colors.LinearSegmentedColormap('cpt_ir', cpt_ir)

# extrai os limites das latitudes e longitudes
lonmin, lonmax, latmin, latmax = -60.0, -40., -36., -24.0

# lista das imagens
files = ['S10635346_202404261600.nc',
         'S10635346_202404271500.nc',
         'S10635346_202404280000.nc',
         'S10635346_202404291600.nc',
         'S10635346_202404300600.nc',
         'S10635346_202405011500.nc',
         'S10635346_202405012200.nc',
         'S10635346_202405020300.nc',
         'S10635346_202405020700.nc',
         'S10635346_202405021200.nc',
         'S10635346_202405030400.nc',
         'S10635346_202405041930.nc']

#==================================================================================================#
#                              DEFINIÇÕES DO GRÁFICO
#==================================================================================================#
# cria moldura da figura
fig, ax = pplt.subplots(ncols=3, nrows=4, axheight=6, tight=True, proj='pcarree')

# formata os eixos
ax.format(coast=True, borders=True, innerborders=False,
          labels=True, latlines=5, lonlines=5,
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax),
          abc=True, abcstyle='(a)', abcsize=5,
          small='40px', large='55px',
          suptitle='GOES-16 CH13 (10.35 µm)')

#==================================================================================================#
#                                     LOOP DAS IMAGENS
#==================================================================================================#
for i, file in enumerate(files):

    #--------------------------------------------------------#
    #                    Leitura ABI
    #--------------------------------------------------------#
    # nome do arquivo
    basename = os.path.basename(os.path.splitext(file)[0]) # S10635346_202405011500.nc

    # extrai a data da imagem
    ano, mes, dia, hor, min = basename[10:14], basename[14:16], basename[16:18], basename[18:20], basename[20:22]

    # leitura do arquivo Netcdf
    imagem = xr.open_dataset(f'dados_abi/{file}').sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))

    # extrai os dados
    data, lat, lon = (imagem['Band1']/100)-273.15, imagem['lat'], imagem['lon']

    # é necessário inverter o eixo y
    data = np.flipud(data)

    #--------------------------------------------------------#
    #                   Plota imagem
    #--------------------------------------------------------#
    if i == 0:
        map1 = ax[i].imshow(data,
                            cmap=cmap_ir, vmin=-103.0, vmax=105,
                            extent=[lonmin, lonmax, latmin, latmax],
                            levels=pplt.arange(-103.0, 105, 1.0))
    else:
        ax[i].imshow(data,
                     cmap=cmap_ir, vmin=-103.0, vmax=105,
                     extent=[lonmin, lonmax, latmin, latmax],
                     levels=pplt.arange(-103.0, 105, 1.0))
        
    # plota titulo de cada figura
    if (i==1 or i==2 or i==4 or i==5 or i==7 or i==8): ax[i].format(title=f'{ano}{mes}{dia} {hor}{min} UTC', labels=False, linewidth=3)
    if (i==10 or i==11): ax[i].format(title=f'{ano}{mes}{dia} {hor}{min} UTC', labels=[False, False, True, False], linewidth=3)
    if (i==0 or i==3 or i==6): ax[i].format(title=f'{ano}{mes}{dia} {hor}{min} UTC', labels=[True, False, False, False], linewidth=3)
    if (i==9): ax[i].format(title=f'{ano}{mes}{dia} {hor}{min} UTC', labels=[True, False, True, False], linewidth=3)

    # plota contornos
    evm_plot_states('https://github.com/evmpython/shapefile/raw/main/UFs/RS/RS_UF_2019.shp', 'black', 1.8)
    evm_plot_states('https://github.com/evmpython/shapefile/raw/main/UFs/SC/SC_UF_2019.shp', 'black', 1.8)
    evm_plot_states('https://github.com/evmpython/shapefile/raw/main/UFs/PR/PR_UF_2019.shp', 'black', 1.8)
    
#==================================================================================================#
#                                   BARRA DE CORES E SALVA FIGURA
#==================================================================================================#
# plota barra de cores
fig.colorbar(map1, loc='b', label='Brightness Temperature ($\degree$C)', ticks=25, ticklabelsize=40, labelsize=40, width=0.6, length=0.5)

# salva figura
fig.save('Fig_4_GOES16_ch13_composicao_proof.jpg', dpi=300)

# **Plota figura** - `COM CÍRCULOS DOS RADARES`

In [ ]:
%%time
#==================================================================================================#
#                              DEFINIÇÃO DE PARÂMETROS
#==================================================================================================#
# corees do IR
cpt_ir = loadCPT('ir.cpt')
cmap_ir = cm.colors.LinearSegmentedColormap('cpt_ir', cpt_ir)

# extrai os limites das latitudes e longitudes
lonmin, lonmax, latmin, latmax = -60.0, -40., -36., -24.0

# lista das imagens
files = ['S10635346_202404261600.nc',
         'S10635346_202404271500.nc',
         'S10635346_202404280000.nc',
         'S10635346_202404291600.nc',
         'S10635346_202404300600.nc',
         'S10635346_202405011500.nc',
         'S10635346_202405012200.nc',
         'S10635346_202405020300.nc',
         'S10635346_202405020700.nc',
         'S10635346_202405021200.nc',
         'S10635346_202405030400.nc',
         'S10635346_202405041930.nc']

#==================================================================================================#
#                              DEFINIÇÕES DO GRÁFICO
#==================================================================================================#
# cria moldura da figura
fig, ax = pplt.subplots(ncols=3, nrows=4, axheight=6, tight=True, proj='pcarree')

# formata os eixos
ax.format(coast=True, borders=True, innerborders=False,
          labels=True, latlines=5, lonlines=5,
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax),
          abc=True, abcstyle='a)', abcsize=5,
          small='40px', large='55px',
          suptitle='GOES-16 CH13 (10.35 µm)')

"""
# plota os aneis de distância do radar
evm_plota_aneis_em_geral_painel([240], -49.47176413, -28.1285956, 'gray', label='Morro da Igreja (SC) Radar', linestyle='solid', axx=0, lww=0.5)
evm_plota_aneis_em_geral_painel([240], -52.60374, -27.04879, 'gray', label='Chapecó (SC) Radar', linestyle='dotted', axx=0, lww=1.5)
evm_plota_aneis_em_geral_painel([240], -52.7016774, -31.4039101, 'gray', label='Canguçu (RS) Radar', linestyle='solid', axx=0, lww=0.5)
evm_plota_aneis_em_geral_painel([240], -54.930064194, -29.22521647222, 'gray', label='Santiago (RS) Radar', linestyle='dotted', axx=0, lww=1.5)
"""

#==================================================================================================#
#                                     LOOP DAS IMAGENS
#==================================================================================================#
for i, file in enumerate(files):

    #--------------------------------------------------------#
    #                    Leitura ABI
    #--------------------------------------------------------#
    # nome do arquivo
    basename = os.path.basename(os.path.splitext(file)[0]) # S10635346_202405011500.nc

    # extrai a data da imagem
    ano, mes, dia, hor, min = basename[10:14], basename[14:16], basename[16:18], basename[18:20], basename[20:22]

    # leitura do arquivo Netcdf
    imagem = xr.open_dataset(f'dados_abi/{file}').sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))

    # extrai os dados
    data, lat, lon = (imagem['Band1']/100)-273.15, imagem['lat'], imagem['lon']

    # é necessário inverter o eixo y
    data = np.flipud(data)

    #--------------------------------------------------------#
    #                   Plota imagem
    #--------------------------------------------------------#
    if i == 0:
        map1 = ax[i].imshow(data,
                            cmap=cmap_ir, vmin=-103.0, vmax=105,
                            extent=[lonmin, lonmax, latmin, latmax],
                            levels=pplt.arange(-103.0, 105, 1.0))
    else:
        ax[i].imshow(data,
                     cmap=cmap_ir, vmin=-103.0, vmax=105,
                     extent=[lonmin, lonmax, latmin, latmax],
                     levels=pplt.arange(-103.0, 105, 1.0))
        
    # plota titulo de cada figura
    if (i==1 or i==2 or i==4 or i==5 or i==7 or i==8): ax[i].format(title=f'{ano}{mes}{dia} {hor}{min} UTC', labels=False, linewidth=3)
    if (i==10 or i==11): ax[i].format(title=f'{ano}{mes}{dia} {hor}{min} UTC', labels=[False, False, True, False], linewidth=3)
    if (i==0 or i==3 or i==6): ax[i].format(title=f'{ano}{mes}{dia} {hor}{min} UTC', labels=[True, False, False, False], linewidth=3)
    if (i==9): ax[i].format(title=f'{ano}{mes}{dia} {hor}{min} UTC', labels=[True, False, True, False], linewidth=3)

    # plota contornos
    evm_plot_states('https://github.com/evmpython/shapefile/raw/main/UFs/RS/RS_UF_2019.shp', 'black', 1.0)
    evm_plot_states('https://github.com/evmpython/shapefile/raw/main/UFs/SC/SC_UF_2019.shp', 'black', 1.0)
    evm_plot_states('https://github.com/evmpython/shapefile/raw/main/UFs/PR/PR_UF_2019.shp', 'black', 1.0)
    
    # plota os aneis de distância do radar
    evm_plota_aneis_em_geral_painel([240], -49.47176413, -28.1285956, 'black', label='Morro da Igreja (SC) Radar', linestyle='solid', axx=i, lww=0.5)
    evm_plota_aneis_em_geral_painel([240], -52.60374, -27.04879, 'black', label='Chapecó (SC) Radar', linestyle='dotted', axx=i, lww=1.5)
    evm_plota_aneis_em_geral_painel([240], -52.7016774, -31.4039101, 'black', label='Canguçu (RS) Radar', linestyle='solid', axx=i, lww=0.5)
    evm_plota_aneis_em_geral_painel([240], -54.930064194, -29.22521647222, 'black', label='Santiago (RS) Radar', linestyle='dotted', axx=i, lww=1.5)
    
#==================================================================================================#
#                                   BARRA DE CORES E SALVA FIGURA
#==================================================================================================#
# plota barra de cores
fig.colorbar(map1, loc='b', label='Brightness Temperature ($\degree$C)', ticks=25, ticklabelsize=40, labelsize=40, width=0.6, length=0.5)

# salva figura
fig.save('Fig_4_GOES16_ch13_composicao_comradares.jpg', dpi=300)